In [9]:
import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np
import time

In [10]:
def compute_pi_cpu(n_points):
    x_rand = np.random.rand(n_points)
    y_rand = np.random.rand(n_points)

    #n_inside = 0
    #for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2 + y_rand[i]**2) <= 1.0
    
    # 1. Vectorize loop
    inside = np.sqrt(x_rand**2+y_rand**2) <= 1.0
    n_inside = np.sum(inside)

    pi = 4 * n_inside/n_points

    return pi

In [11]:
tic = time.time()
print(compute_pi_cpu(1024))
toc = time.time()

print("Time to execute cpu version: {:f} seconds".format(toc-tic))

3.23046875
Time to execute cpu version: 0.000640 seconds


In [12]:
pi_kernel_src = """
//Based on Stroustrup, adapted for CUDA
//pseudorandom numbers
__device__ float generateRandomNumber(long& last_draw) {
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0; 
}

__global__ void computePi(unsigned int* inside, unsigned int seed) {
    //1 generate random numbers
    //unsigned int tid = threadIdx.x;
    unsigned int tid = blockIdx.x*blockDim.x + threadIdx.x;
    
    long rand_seed = seed + tid;
    float x = generateRandomNumber(rand_seed);
    float y = generateRandomNumber(rand_seed);
    
    //2 compute the radius from the origin
    float r = sqrt(x*x + y*y);
    
    //3 check if it is inside the circle
    if (r <= 1) {
        inside[tid] = 1;
    }
}
"""

mod = SourceModule(pi_kernel_src)
func = mod.get_function("computePi")

In [24]:
def compute_pi_gpu(n_points, threads_per_block=512):
    assert(n_points % threads_per_block == 0)
    
    bytes_per_uint = 4
    
    inside_gpu = cuda.mem_alloc(bytes_per_uint * n_points)
    
    num_blocks = n_points // threads_per_block
    func(inside_gpu, np.uint32(time.time()), block=(threads_per_block,1,1), grid=(num_blocks,1,1))
    
    inside_cpu = np.empty(n_points, dtype=np.uint32)
    cuda.memcpy_dtoh(inside_cpu, inside_gpu)
    
    n_inside = np.sum(inside_cpu)
                
    pi = 4 * n_inside/n_points
            
    return pi

In [26]:
tic = time.time()
print(compute_pi_gpu(512000000))
toc = time.time()

print("Time to execute gpu version: {:f} seconds".format(toc-tic))

3.1415915703125
Time to execute gpu version: 1.403937 seconds
